# OpenSearch Hybrid 검색을 통한 RAG
> 이 노트북은  SageMaker Studio* **`Data Science 3.0`** kernel 및 ml.t3.medium 인스턴스에서 테스트 되었습니다.
---
### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

### 선수조건
- 이 노트북은 이전 노트북인 "02_KR_RAG_OpenSearch_Claude.ipynb" 이 완료 되었다고 가정 합니다.
    - 오픈서치 인텍스 관련 정보를 참조 합니다.

### Methods and Resources for Hybrid search with Re-ranking 
- Score Normalization
    - [MinMax based](https://towardsdatascience.com/text-search-vs-vector-search-better-together-3bd48eb6132a)
    - [z-socre based](https://towardsdatascience.com/hybrid-search-2-0-the-pursuit-of-better-search-ce44d6f20c08)
- Reciprocal Rank Fusion (RRF)
    - [Paper](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf)
    - [Description](https://medium.com/@sowmiyajaganathan/hybrid-search-with-re-ranking-ff120c8a426d)
- [LangChain API for Ensemble Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble)

### 설정

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) Bedrock에 연결해야 합니다.



In [2]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

# 1. Bedrock Client 생성

In [3]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


# 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

## LLM 로딩 (Claude-v2)

In [4]:
from langchain.llms.bedrock import Bedrock

In [5]:
# - create the Anthropic Model
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=boto3_bedrock,
    model_kwargs={'max_tokens_to_sample': 512}
)
llm_text

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f1639f40550>, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 512})

## Embedding 모델 선택

In [6]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):
    if is_bedrock_embeddings:

        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings
        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
            )
        )
        print("Bedrock Embeddings Model Loaded")

    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )        
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")
    
    return llm_emb

#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [7]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)    

Bedrock Embeddings Model Loaded


# 3. LangChain OpenSearch VectorStore 생성 
## 선수 조건
- 이전 노트북 02_1_KR_RAG_OpenSearch_Keyword.ipynb 또는 02_1_KR_RAG_OpenSearch_Semantic.ipynb를 통해서 OpenSearch Index 가 생성이 되어 있어야 합니다.

## 오픈 서치 도메인 및 인증 정보 세팅

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

#### [중요] 아래에 OpenSearch ID/PW 를 입력을 해주세요.

In [8]:
os.environ["OpenSearch_UserName"] = "Type ID"
os.environ["OpenSearch_UserPassword"] = "Type PW"


rag_user_name = os.environ["OpenSearch_UserName"]
rag_user_password = os.environ["OpenSearch_UserPassword"]

In [9]:
opensearch_domain_endpoint = "<Type your domain endpoint>"


http_auth = (rag_user_name, rag_user_password) # Master username, Master password

index_name = "genai-demo-index-v1-with-tokenizer"

## LangChain OpenSearch VectorStore 생성

In [10]:
from langchain.vectorstores import OpenSearchVectorSearch

In [11]:
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2"
)

## OpenSearch Client 생성

In [12]:
from utils.opensearch import opensearch_utils

In [13]:
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 은전한잎 형태소 분석기 (seunjeon_tokenizer) 사용하기 in Opensearch
- 형태소 분석기에 대한 자세한 사항은 02_1_KR_RAG_OpenSearch_Keyword.ipynb 참고

### 인덱스 확인 (tokenization 확인)

In [14]:
index_info = os_client.indices.get(index=index_name)
pprint(index_info)

{'genai-demo-index-v1-with-tokenizer': {'aliases': {},
                                        'mappings': {'properties': {'metadata': {'properties': {'row': {'type': 'long'},
                                                                                                'source': {'fields': {'keyword': {'ignore_above': 256,
                                                                                                                                  'type': 'keyword'}},
                                                                                                           'type': 'text'},
                                                                                                'timestamp': {'type': 'float'},
                                                                                                'type': {'fields': {'keyword': {'ignore_above': 256,
                                                                                                                         

# 4. 오픈 서치에 "유사 서치" 검색
- query 를 제공해서 실제로 유사한 내용이 검색이 되는지를 확인 합니다.



- similarity_search_with_score API 정보
    - [API: similarity_search_with_score](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html#langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.similarity_search)

In [15]:
import copy
from langchain.schema import Document
from langchain import PromptTemplate
from operator import itemgetter

## (1) OpenSearch Vector 검색

### 프로프트 템플릿 생성

In [16]:
from utils.rag import run_RetrievalQA, show_context_used
from langchain.prompts import PromptTemplate

### [TIP] Prompt의 instruction의 경우 한글보다 **영어**로 했을 때 더 좋은 결과를 얻을 수 있습니다.

In [82]:
# prompt_template = """
# \n\nHuman: 다음 문맥의 Information을 사용하여 고객 서비스 센터 직원처럼, 마지막 질문에 대한 목차 형식으로 답변을 제공하세요. 응답을 모르면 모른다고 말하고 응답을 만들려고 하지 마세요.

# {context}

# Question: {question}
# \n\nAssistant:"""

prompt_template = """


Human: Here is the context, inside <context></context> XML tags.    

<context>
{context}
</context>

Only using the contex as above, answer the following question with the rules as below:
    - Don't insert XML tag such as <context> and </context> when answering.
    - Write as much as you can
    - Be courteous and polite
    - Only answer the question if you can find the answer in the context with certainty.

Question:
{question}

If the answer is not in the context, just say "주어진 내용에서 관련 답변을 찾을 수 없습니다."


Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


### 필터 생성

In [83]:
filter01 = "인터넷뱅킹"
# filter01 = "인증서"
filter02 = "신한은행"
# filter02 = "아마존은행"

query = "홈페이지 이용자아이디 여러 개 사용할 수 있나요?"
# query = "타기관OTP 등록 방법 알려주세요"

boolean_filter = opensearch_utils.get_filter(
    filter=[
        {"term": {"metadata.type": filter01}},
        {"term": {"metadata.source": filter02}},
    ]
)

pprint(boolean_filter)

{'bool': {'filter': [{'term': {'metadata.type': '인터넷뱅킹'}},
                     {'term': {'metadata.source': '신한은행'}}]}}


### LangChain RetrievalQA 를 통해 실행
- 아래와 같이 top_k = 5 를 실행하면 해당 관련 Context 를 가져오지 못하고 , "주어진 내용에서 찾을 수 없습니다" 라고 답변 함.

In [85]:
result = run_RetrievalQA(
    query=query,
    boolean_filter=boolean_filter,
    llm=llm_text,
    prompt=PROMPT,
    vector_db=vector_db,
    verbose=True,
    k=5
)

print("##################################")
print("query: ", query)
print("boolean_filter: ", boolean_filter)
print("##################################")

print (colored("\n\n### Answer ###", "blue"))
print_ww(result['result'])

print (colored("\n\n### Contexts ###", "green"))
show_context_used(result['source_documents'])



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human: Here is the context, inside <context></context> XML tags.    

<context>
ask: 개인정보 변경은 어디서 해야 하나요?
Information: 인터넷뱅킹에 로그인 하신 후 "사용자관리"를 클릭하신 후 "고객정보" > "고객정보조회/변경" 에서 개인정보를 변경하실 수 있습니다.

ask: 이체한도란 무엇인가요?
Information: 이체한도란 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등을 통하여 이체할 수 있는 거래한도를 말합니다. ① 1일/1회 이체한도 1일 이체한도란 하루에 거래할 수 있는 이체금액의 합을 말하여, 1회 이체한도란 1회에 이체하실 수 있는 금액의 한도를 말합니다. ② 통합이체한도 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등을 고객님이 사용하시는 경우 고객님이 1일 또는 1회에 이체하실 수 있는 이체한도를 말합니다. 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등에서 고객님이 거래하신 이체금액의 합이 통합이체한도의 범위를 초과할 수 없습니다. ③ 최고이체한도 금융감독원에서는 개인의 전자금융 시 이용할 수 있는 이체한도의 가이드라인을 정하고 있습니다.(인터넷뱅킹 : 1일 5억원, 1회 1억원 이내)

ask: 해외고객 추가인증 방법
Information: 먼저 이용에 불편을 드려 대단히 죄송합니다. 2013년 9월 전자금융사기 예방서비스 전면 시행으로 인해, 국내외에서 뱅킹이용시 거래에 따라 추가인증을 거치게 되었습니다.
① 인증서 발급/재발급, 1일 누적 300만원 이상 이체시 시행하는 추가인증 방법(누적금액은 변경될수 있습니다.)해외IP로 접속하신 부분이 자동으로 확인되면 ‘해외체류 확인’ 항목이 활성화 되기 때문에 법무부 출입국관리사무소에 출

## (2) OpenSearch Keyword 검색
- "minimum_should_match=50" 세팅을 하고, 관련 컨텍스트를 찾아 오지 못합니다.

In [91]:
from utils.opensearch import opensearch_utils
from utils.rag import get_keyword_similar_docs 
from langchain.chains.question_answering import load_qa_chain

In [92]:
filter01 = "인터넷뱅킹"
# filter01 = "인증서"
filter02 = "신한은행"
# filter02 = "아마존은행"

query = "홈페이지 이용자아이디 여러 개 사용할 수 있나요?"
#query = "타기관OTP 등록 방법 알려주세요"

search_keyword_result = get_keyword_similar_docs(
    query=query,
    minimum_should_match=50,
    filter=[
        {"term": {"metadata.type": filter01}},
        {"term": {"metadata.source": filter02}},
    ],
    index_name=index_name,
    os_client=os_client,
    k=5,
    hybrid=False
)
print(search_keyword_result)

keyword search query: 
{'query': {'bool': {'filter': [{'term': {'metadata.type': '인터넷뱅킹'}},
                               {'term': {'metadata.source': '신한은행'}}],
                    'must': [{'match': {'text': {'minimum_should_match': '50%',
                                                 'operator': 'or',
                                                 'query': '홈페이지 이용자아이디 여러 개 '
                                                          '사용할 수 있나요?'}}}]}},
 'size': 5}
[Document(page_content='ask: 인터넷뱅킹 비밀번호를 잊어버렸습니다.\nInformation: 인터넷뱅킹 로그인시 사용되는 인증서 암호를 잊어버리셨다면, 영업점 방문없이 인증서 재발급을 통해 인증서 암호를 다시 설정하실수 있습니다. 당행에서 발급한 인증서인 경우 당행 홈페이지에서 재발급하시면 됩니다. 신한은행 홈페이지 인증센터로 접속하시면 금융인증서와 공동인증서구 공인인증서로 메뉴가 분리되어 있으므로 해당 메뉴를 선택해서 접속해 주시면 됩니다. 타기관에서 발급된 인증서인 경우엔 해당기관에서 재발급후, 당행 홈페이지 타기관 공인인증서 등록/해제 메뉴에서 등록 후 이용하시면 됩니다.', metadata={'source': '신한은행', 'row': 71, 'type': '인터넷뱅킹', 'timestamp': 1696832766.2919788, 'id': '601b677c-b2dc-4e78-9dd5-cfa791c4c5d6'})]


In [93]:
chain = load_qa_chain(
    llm=llm_text,
    chain_type="stuff",
    prompt=PROMPT,
    verbose=True
)

answer = chain.run(
    input_documents=search_keyword_result,
    question=query
)

print("##############################")
print("query: \n", query)
print("answer: \n", answer)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human: Here is the context, inside <context></context> XML tags.    

<context>
ask: 인터넷뱅킹 비밀번호를 잊어버렸습니다.
Information: 인터넷뱅킹 로그인시 사용되는 인증서 암호를 잊어버리셨다면, 영업점 방문없이 인증서 재발급을 통해 인증서 암호를 다시 설정하실수 있습니다. 당행에서 발급한 인증서인 경우 당행 홈페이지에서 재발급하시면 됩니다. 신한은행 홈페이지 인증센터로 접속하시면 금융인증서와 공동인증서구 공인인증서로 메뉴가 분리되어 있으므로 해당 메뉴를 선택해서 접속해 주시면 됩니다. 타기관에서 발급된 인증서인 경우엔 해당기관에서 재발급후, 당행 홈페이지 타기관 공인인증서 등록/해제 메뉴에서 등록 후 이용하시면 됩니다.
</context>

Only using the contex as above, answer the following question with the rules as below:
    - Don't insert XML tag such as <context> and </context> when answering.
    - Write as much as you can
    - Be courteous and polite
    - Only answer the question if you can find the answer in the context with certainty.

Question:
홈페이지 이용자아이디 여러 개 사용할 수 있나요?

If the answer is not in the context, just say "주어진 내용에서 관련 답변을 찾을 수 없습니다."


Assistant:

> Finished chain.

> Finished chain.
######

### 키워드 검색 결과 (search_keyword_result)
bm25 score는 max_value로 normalization 되어 있음 (score range 0 - 1)

In [94]:
search_keyword_result

[Document(page_content='ask: 인터넷뱅킹 비밀번호를 잊어버렸습니다.\nInformation: 인터넷뱅킹 로그인시 사용되는 인증서 암호를 잊어버리셨다면, 영업점 방문없이 인증서 재발급을 통해 인증서 암호를 다시 설정하실수 있습니다. 당행에서 발급한 인증서인 경우 당행 홈페이지에서 재발급하시면 됩니다. 신한은행 홈페이지 인증센터로 접속하시면 금융인증서와 공동인증서구 공인인증서로 메뉴가 분리되어 있으므로 해당 메뉴를 선택해서 접속해 주시면 됩니다. 타기관에서 발급된 인증서인 경우엔 해당기관에서 재발급후, 당행 홈페이지 타기관 공인인증서 등록/해제 메뉴에서 등록 후 이용하시면 됩니다.', metadata={'source': '신한은행', 'row': 71, 'type': '인터넷뱅킹', 'timestamp': 1696832766.2919788, 'id': '601b677c-b2dc-4e78-9dd5-cfa791c4c5d6'})]

## (3) OpenSearch Hybrid 검색
- 아래는 결론적으로 query = "홈페이지 이용자아이디 여러 개 사용할 수 있나요?" 에 답변을 제공 합니다.
    - 시멘티 서치에서 LangChain 의 RetrievalQA 사용 대신에 , LangChain OpenSearch Vector Store의 similarity_search_with_score 를 사용 하여 제일 첫번재에 해당 답변이 검색이 됩니다.
    - 키워드 검색에서는 해당 qeury 가 검색이 되지 않습니다.
    - 이후에 시멘틱 검색, 키워드 검색에 가중치를 [0.3 , 0.5] 를 주어서 Ensembe 을 하여 랭킹을 하면, top_k = 5 에서 5번재로 Context 로 포함이 되어서 , 최종 검색이 됩니다.

#### OpenSearch Hybrid 는 아래와 같은 방식으로 작동합니다.
- (1) "Vector 서치" 하여 스코어를 얻은 후에 표준화를 하여 스코어를 구함. 
    - 전체 결과에서 가장 높은 스코어는 표준화 과정을 통하여 스코어가 1.0 이 됨.
- (2) Keyword 서치도 동일하게 함.
- (3) 위의 두 개의 결과에 랭킹 알고리즘 ("RRF" 혹은 "simple_weighted"] 을 통해, 스코어를 계산하여 정렬하여 제공 함.
    - 여기서는 simple_weighted 사용 함.

스코어 표준화 예시

<pre>
Docs    Score	    Normalized-Score

Doc1: 	0.0083		1.0

Doc2: 	0.0074		0.8900

Doc3: 	0.0071		0.8585

Example: 0.0074 / 0.0083 = 0.8900
</pre>

In [36]:
from utils.rag import interpolate_results, sort_score_search_results
from utils.rag import get_semantic_similar_docs, get_keyword_similar_docs, ensemble_results#, get_similiar_docs

In [101]:
def search_hybrid(**kwargs):
    
    assert "query" in kwargs, "Check your query"
    assert "vector_db" in kwargs, "Check your vector_db"
    assert "index_name" in kwargs, "Check your index_name"
    assert "os_client" in kwargs, "Check your os_client"
    assert "qa_chain" in kwargs, "Check your qa_chain"

    similar_docs_semantic = get_semantic_similar_docs(
        vector_db=kwargs["vector_db"],
        query=kwargs["query"],
        k=kwargs.get("k", 5),
        hybrid=True
    )
    print("##############################")
    print("similar_docs_semantic")
    print("##############################")    
    print(similar_docs_semantic)    

    similar_docs_keyword = get_keyword_similar_docs(
        query=query,
        minimum_should_match=kwargs.get("minimum_should_match", 0),
        filter=kwargs.get("filter", []),
        index_name=kwargs["index_name"],
        os_client=kwargs["os_client"],
        k=kwargs.get("k", 5),
        hybrid=True
    )
    
    print("##############################")    
    print("similar_docs_keyword")    
    print("##############################")    
    print(similar_docs_keyword)        

    similar_docs_ensemble = ensemble_results(
        doc_lists = [similar_docs_semantic, similar_docs_keyword],
        weights = kwargs.get("ensemble_weights", [.5, .5]),
        algorithm="simple_weighted",
        c=60,
        k=kwargs.get("k", 5)
    )

    print("##############################")
    print("similar_docs_ensemble")
    print("##############################")    
    print(similar_docs_ensemble)        
    
    similar_docs_ensemble = list(map(lambda x:x[0], similar_docs_ensemble))

    
    answer = kwargs["qa_chain"].run(
        input_documents=similar_docs_ensemble,
        question=query
    )

    return answer


In [102]:
%%time

filter01 = "인터넷뱅킹"
# filter01 = "인증서"
filter02 = "신한은행"
# filter02 = "아마존은행"

query = "홈페이지 이용자아이디 여러 개 사용할 수 있나요?"
#query = "타기관OTP 등록 방법 알려주세요"
#query = "웹싸이트 계정몇 개 사용할 수 있나요?"

answer = search_hybrid(
    query=query,
    vector_db=vector_db,
    k=5,
    index_name=index_name,
    os_client=os_client,
    filter=[
        {"term": {"metadata.type": filter01}},
        {"term": {"metadata.source": filter02}},
    ],
    ensemble_weights=[.3, .5], # 시멘트 서치에 가중치 0.3 , 키워드 서치 가중치 0.5 부여.
    qa_chain=chain
)

##############################
similar_docs_semantic
##############################
[(Document(page_content='ask: 홈페이지 이용자아이디 여러 개 사용할 수 있나요?\nInformation: 홈페이지 이용자 아이디는 개인의 경우 1인 1개만 이용 가능하고 기업의 경우에는 1개의 사업자번호당 사용자별로 이용자아이디를 복수로 사용할 수 있습니다. \r\n※ 개인사업자의 경우 개인과 기업 각각 이용자아이디를 발급하여 복수로 이용 가능합니다. \r\n기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.', metadata={'source': '신한은행', 'row': 21, 'type': '홈페이지', 'timestamp': 1696832766.2918196}), 1.0), (Document(page_content='ask: 홈페이지 회원아이디를 변경하고 싶습니다.\nInformation: 회원탈퇴를 하시고, 사용하시고 싶은 아이디로 신규 회원가입신청을 하시면 됩니다. 다만, 사용하시고 싶은 아이디가 이미 다른 고객이 사용하고 있는 경우에는 사용하실 수 없습니다. 기타 문의는 콜센터 1599-8000번으로 문의 바랍니다.', metadata={'source': '신한은행', 'row': 45, 'type': '홈페이지', 'timestamp': 1696832766.2918956}), 0.8344235482494401), (Document(page_content="ask: 만14세 미만의 고객은 홈페이지 회원가입이 가능하나요?\nInformation: 만 14세 미만 고객은 회원가입시 '정보통신망 이용촉진 및 정보 등에 관한 법률' 및 '개인정보보호지침'에 따라 법정대리인의 정보활용 동의가 필요합니다. 당행 영업점을 통한 보호자분의 홈페이지 회원 법정대리인 등록이 완료된 고객님에 한하여 회원등록이 가능하므로 홈페이지 회원 법정대리인 등록이 안된 고객님

In [70]:
print (f'question: {query}')
print (f'response: {answer}')

question: 홈페이지 이용자아이디 여러 개 사용할 수 있나요?
response:  주어진 내용에서,

홈페이지 이용자 아이디는 개인의 경우 1인 1개만 이용 가능하고 기업의 경우에는 1개의 사업자번호당 사용자별로 이용자아이디를 복수로 사용할 수 있습니다.
개인사업자의 경우 개인과 기업 각각 이용자아이디를 발급하여 복수로 이용 가능합니다.

따라서 개인의 경우 1개만 사용 가능하나, 기업이나 개인사업자의 경우 복수 사용이 가능함을 알 수 있습니다.
